## Run Tutorial Kernel

This section inserts all the code fragments from the prior notebook  
Then we'll build the gcc and clang versions of the code in new build directories
Then plot some results

[Back to Table of Contents](./00-intro-and-contents.ipynb)

In [ ]:
%%bash
# Run this cell to perform some setup to modify the RAJAPerf source to accommodate the new kernel
pushd $HOME/scripts
chmod +x insert_kernels_into_skeletons.sh
chmod +x restore_skeletons.sh
./insert_kernels_into_skeletons.sh
mkdir -p $HOME/code/RAJAPerf/build_tut_gcc
mkdir -p $HOME/code/RAJAPerf/build_tut_clang
popd
# note you can restore the state of RAJAPerf skeleton files by exiting to a terminal session
# pushd to ~/scripts directory
# run restore_skeleton.sh
# or add a new %%bash cell below and do the same

Before we inserted code snippets we performed the following
copy over the skeletal files and setup a CMakeLists.txt in the tutorial kernel directory
```
cp $HOME/skeletal/TUTORIAL_KERNEL_SKELETON-omp.cpp $TUTKERNEL_DIR/TUTORIAL_KERNEL-omp.cpp
cp $HOME/skeletal/TUTORIAL_KERNEL_SKELETON-seq.cpp $TUTKERNEL_DIR/TUTORIAL_KERNEL-seq.cpp
cp $HOME/skeletal/TUTORIAL_KERNEL_SKELETON.cpp $TUTKERNEL_DIR/TUTORIAL_KERNEL.cpp
cp $HOME/skeletal/TUTORIAL_KERNEL_SKELETON.hpp $TUTKERNEL_DIR/TUTORIAL_KERNEL.hpp
cp $HOME/skeletal/src_tutor_CMakeLists.txt $TUTKERNEL_DIR/CMakeLists.txt
```

with CMakeLists.txt
```
blt_add_library(
  NAME tutkernel
  SOURCES
  TUTORIAL_KERNEL.cpp
  TUTORIAL_KERNEL-seq.cpp
  TUTORIAL_KERNEL-omp.cpp
  DEPENDS_ON common ${RAJA_PERFSUITE_DEPENDS}
  )
```

In [ ]:
%%bash
# Run this cell to compile the new code for gcc
pushd ~/code/RAJAPerf/build_tut_gcc/
cmake -C ../hc*gcc* ..
make -j 5
popd

In [ ]:
%%bash
# Run this cell to compile the new code for clang
pushd ~/code/RAJAPerf/build_tut_clang/
cmake -C ../hc*clang* ..
make -j 5
popd

In [ ]:
%%bash
mkdir -p $HOME/data/sweeps_tut_kernel && pushd $HOME/data/sweeps_tut_kernel
chmod +x $HOME/code/RAJAPerf/scripts/sweep/sweep_size.sh
$HOME/code/RAJAPerf/scripts/sweep/sweep_size.sh -x sweep_gcc $HOME/code/RAJAPerf/build_tut_gcc/bin/raja-perf.exe --size-min 1024 --size-max 1000000 --size-ratio 2 -- --checkrun 3 --kernels Tutorial
$HOME/code/RAJAPerf/scripts/sweep/sweep_size.sh -x sweep_clang $HOME/code/RAJAPerf/build_tut_clang/bin/raja-perf.exe --size-min 1024 --size-max 1000000 --size-ratio 2 -- --checkrun 3 --kernels Tutorial
echo "All done with sweeps!"

The following cell will generate a graph (.png) of problem size vs time, and it can be found by navigating to the data/sweeps_tut_kernel directory

In [ ]:
%%bash
pushd $HOME/data/sweeps_tut_kernel
/opt/conda/bin/python3 $HOME/code/RAJAPerf/scripts/sweep/sweep_graph.py  -d sweep_gcc sweep_clang --caliper -v RAJA_Seq RAJA_OpenMP -slg "time(ms)" -o graph
echo "All done with graphs!"